In [1]:
import pandas as pd
import os
import pickle
import numpy as np

In [2]:
def get_contact_file():
    contact_dir = "../../contact_data/contact/"
    contact_file_list = os.listdir(contact_dir)
    if contact_file_list.__contains__(".DS_Store"):
        contact_file_list.remove(".DS_Store")
    return contact_dir, contact_file_list

def open_contact(contact_dir, contact_file):
    with open(contact_dir+contact_file, 'rb') as f:
        contact = pickle.load(f)
    return contact



In [3]:
contact_dir, contact_file_list = get_contact_file()
contact = open_contact(contact_dir, contact_file_list[1])

In [4]:
t = set()
for kk in contact.keys():
    for k, v in contact[kk].items():
        t.update(v)
tt = max(t)
tt/60/60

25.46527777777778

In [5]:
a = ['a', 'b']
b = ['q', 'c']
c = [-1, -1]
d = {'w':a, 'ww':b, 'ee':c}
pd.DataFrame(d)

,w,ww,ee
0,a,q,-1
1,b,c,-1


In [6]:
rating = pd.read_csv("../../contact_data/ratings25.csv")

In [7]:
users = rating["userId"].drop_duplicates().values

In [8]:
count = rating["userId"].value_counts()

In [9]:
count.index

Int64Index([ 72315,  80974, 137293,  33844,  20055, 109731,  92046,  49403,
             30879, 115102,
            ...
            119415,  29156,  29161,  29164, 119501,  12094, 119539, 156759,
             12084,  36207],
           dtype='int64', length=162541)

In [10]:
c = count.values

In [11]:
c[c>1000].size, c[c>800].size, c[c>600].size, c[c>400].size, c[c>200].size, c[c>150].size, c[c>100].size, c[c>50].size, c[c>40].size, c[c>30].size, c[c>=20].size

(2670, 4177, 7038, 13606, 32848, 43645, 63102, 101074, 114242, 131696, 162541)

In [12]:
threshold_ratings = count.values
large_rating = count.index[:threshold_ratings[threshold_ratings>400].size]
small_rating = count.index[:threshold_ratings[threshold_ratings>400].size]

In [13]:
a = ['a', 'b']
pd.DataFrame({'ww':a, 'ee':[0]*len(a)})

,ww,ee
0,a,0
1,b,0


In [14]:
# def selectUserFromRating(rating, contact):
#     large_rating, small_rating = split_rating(rating)
#     # users1 = large_rating["userId"].drop_duplicates().values
#     # users2 = small_rating["userId"].drop_duplicates().values
#     peers = list(contact.keys())
#     peers1 = int(len(peers) * 0.3)
#     peers2 = len(peers) - peers1
#     selected_users1 = np.random.choice(large_rating, peers1, replace=False)
#     selected_users2 = np.random.choice(small_rating, peers2, replace=False)
#     selected_users = selected_users1.extend(selected_users2)
#     pd.DataFrame({"peerId":peers, "userId":selected_users, "stride":[0]*len(peers), "time":[0]*len(peers), "svd_mse":[0]*len(peers), "mf_svd":[0]*len(peers)})

#     user_peer_dict = {peers[i] : selected_users[i] for i in range(len(selected_users))}
#     user_contact = {i:dict() for i in range(1, 28)}
#     mm = set()
#     for person in contact.keys():
#         peer = user_peer_dict[person]
#         for p, times in contact[person].items():
#             if user_peer_dict.__contains__(p):
#                 for t in times:
#                     n = int(t / (60 * 60)) + 1
#                     mm.add(n)
#                     if user_contact[n].__contains__(peer):
#                         if user_contact[n][peer] is None:
#                             user_contact[n][peer] = {user_peer_dict[p]}
#                         else:
#                             user_contact[n][peer].add(user_peer_dict[p])
#                     else:
#                         user_contact[n][peer] = {user_peer_dict[p]}
#             else:
#                 print(p)
#     return user_contact, selected_users

In [15]:
def split_rating(rating):
        count = rating["userId"].value_counts()
        threshold_ratings = count.values
        large_rating = count.index[:threshold_ratings[threshold_ratings>400].size]
        small_rating = count.index[threshold_ratings[threshold_ratings>400].size:]
        return large_rating, small_rating

In [68]:
#  add stride here
def select_user_for_peer(rating, contact):
    large_rating, small_rating = split_rating(rating)
    peers = list(contact.keys())
    peers1 = int(len(peers) * 0.3)
    peers2 = len(peers) - peers1
    selected_users1 = list(np.random.choice(large_rating, peers1, replace=False))
    selected_users2 = list(np.random.choice(small_rating, peers2, replace=False))
    selected_users = selected_users1 + selected_users2
    return pd.DataFrame({"peerId":peers, "userId":selected_users, "stride":[0]*len(peers), "time":[0]*len(peers), "svd_mse":[0]*len(peers), "mf_svd":[0]*len(peers), 'util':['']*len(peers)}, index=peers)


In [69]:
result = select_user_for_peer(rating, contact)

In [70]:
result.loc['6184901'].at['userId']

145076

In [71]:
def contact_in_hour(result):
    user_contact = {i:dict() for i in range(1, 30)}
    for person in contact.keys():
        peer = result.loc[person].at['userId']
        for p, times in contact[person].items():
            for t in times:
                n = int(t / (60 * 60)) + 1
                if user_contact[n].__contains__(peer):
                    if user_contact[n][peer] is None:
                        user_contact[n][peer] = {result.loc[p].at['userId']}
                    else:
                        user_contact[n][peer].add(result.loc[p].at['userId'])
                else:
                    user_contact[n][peer] = {result.loc[p].at['userId']}        
    return user_contact

In [72]:
result

,peerId,userId,stride,time,svd_mse,mf_svd,util
6184901,6184901,145076,0,0,0,0,
6057201,6057201,7784,0,0,0,0,
212100901,212100901,21100,0,0,0,0,
193308901,193308901,45437,0,0,0,0,
357587601,357587601,116250,0,0,0,0,
...,...,...,...,...,...,...,...
314311201,314311201,115586,0,0,0,0,
freight_2250_berlin-142698,freight_2250_berlin-142698,72262,0,0,0,0,
freight_2250_berlin-142698_freight,freight_2250_berlin-142698_freight,16639,0,0,0,0,
freight_2547_berlin-130736,freight_2547_berlin-130736,43493,0,0,0,0,


In [65]:
a = result.loc[result['userId']==126886, 'util'].values[0]
b = a + '_12'
result.loc[result['userId']==126886, 'util'] = b
result.loc[result['userId']==126886, 'util']

6184901    b_12
Name: util, dtype: object

In [71]:
user_contact = contact_in_hour(result)

In [74]:
if result.loc[result['userId']==145076, 'time'].values[0] == 0:
    print("aaa") 



aaa


In [72]:
result['userId'].value_counts()

10848     1
102725    1
119347    1
63269     1
107228    1
         ..
49986     1
50626     1
44952     1
84582     1
75520     1
Name: userId, Length: 8628, dtype: int64

In [98]:
result[result['userId'] == '10848', 'time'].values[0]
# result.loc['userId' =='10848', 'svd_mse'].values

TypeError: '(6184901                               False
6057201                               False
212100901                             False
193308901                             False
357587601                             False
                                      ...  
314311201                             False
freight_2250_berlin-142698            False
freight_2250_berlin-142698_freight    False
freight_2547_berlin-130736            False
freight_2547_berlin-130736_freight    False
Name: userId, Length: 8628, dtype: bool, 'time')' is an invalid key

In [95]:
result[result['userId'] == '10848']

,peerId,userId,stride,time,svd_mse,mf_svd,util


In [92]:
result.loc[result['userId'] == '10848']

Series([], Name: svd_mse, dtype: int64)

In [87]:

result.loc[result['userId'] == '10848', 'util'].values

array([], dtype=object)

In [86]:
result[result['userId'] == '10848']['util'].values

TypeError: 'numpy.ndarray' object is not callable

In [49]:
result.loc[result['userId'] == 33681]['svd_mse']

6184901    2.0
Name: svd_mse, dtype: float64

In [126]:
sum = 0
for kk in user_contact.keys():
    for k, v in user_contact[kk].items():
        sum += len(v)
sum

15402